In [52]:
import os
import pathlib
import geopandas as gpd

import pandas as pd
import rioxarray as rxr

# Import packages
import numpy as np
import netCDF4
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns

%pip_install netCDF



UsageError: Line magic function `%pip_install` not found.


In [43]:
%store -r comanche_gdf

In [ ]:
model_name = 'BNU-ESM'
variable_name = 'pr'
var_long_name = 'precipitation'
dir_path = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/'

In [44]:
# build a directory for the macav2 data

comanche_path = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data',
    'habitat_suitability'
)
os.makedirs(comanche_path, exist_ok=True)
# comanche_gdf.to_file('habitat_suitability')
# comanche_gdf = gpd.read_file(comanche_path)

# Define the file path for the GeoDataFrame
comanche_file = os.path.join(comanche_path, 'comanche_data.shp')

# Save the GeoDataFrame to the file
comanche_gdf.to_file(comanche_file)

# Read the GeoDataFrame back from the file
comanche_gdf = gpd.read_file(comanche_file)


In [45]:
maca_url = 'http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/BNU-ESM/macav2metdata_pr_BNU-ESM_r1i1p1_rcp85_2096_2099_CONUS_monthly.nc'
xr.open_dataset(maca_url).squeeze().precipitation


OSError: [Errno -90] NetCDF: file not found: 'http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/BNU-ESM/macav2metdata_pr_BNU-ESM_r1i1p1_rcp85_2096_2099_CONUS_monthly.nc'

In [55]:
def convert_longitude(longitude):
        """Convert longitude from 0-360 to -180-180"""
        return (longitude - 360) if longitude > 180 else longitude

maca_da_list = []
for site_name, site_gdf in {'comanche': comanche_gdf}.items():
        for variable in ["pr"]:
                # for start_year in [2091, 2006]:
                #         end_year = start_year + 4
                        maca_url = (
                                "http://thredds.northwestknowledge.net:8080/thredds"
                                "/dodsC/MACAV2/BNU-ESM/macav2metdata_pr_BNU-ESM_"
                                "r1i1p1_rcp85_2096_2099_CONUS_monthly.nc"
                                # "http://thredds.northwestknowledge.net:8080/thredds/"
                                # "fileServer/MACAV2/BNU-ESM/macav2metdata_pr_BNU-ESM_"
                                # "r1i1p1_rcp45_2091_2095_CONUS_monthly.nc"
                                # 'http://thredds.northwestknowledge.net:8080'
                                # '/thredds/fileServer/MACAV2/BNU-ESM/macav2metdata'
                                # '_pr_BNU-ESM_r1i1p1_rcp85_2096_2099_CONUS_monthly.nc'
                        # "http://thredds/fileServer/MACAV2/BNU-ESM/macav2metdata"
                        # f"_{variable}_BNU-ESM_r1i1p1_rcp45_{start_year}"
                        # f"_{end_year}_CONUS_monthly.nc"
                        )
                        maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                        bounds = site_gdf.to_crs(maca_da.rio.crs).total_bounds
                        maca_da = maca_da.assign_coords(
                                lon=("lon", [convert_longitude(l) for l in maca_da.lon.values])
                        )
                        maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
                        maca_da.rio.clip_box(*bounds)
                        maca_da_list.append(dict(
                                site_name=site_name,
                                variable=variable,
                                start_year=start_year,
                                da=maca_da))
pd.DataFrame(maca_da_list)


,site_name,variable,start_year,da
0,comanche,pr,2091,[[[<xarray.DataArray 'precipitation' ()> Size:...


In [ ]:
maca_da.rio.clip_box(*comanche_gdf.to_crs(maca_ds.rio.crs).total_bounds)
maca_da.rio.clip_box(*bounds)

MissingSpatialDimensionError: x dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'x' can address this. Data variable: precipitation

In [ ]:
def convert_longitude(longitude):
        return (longitude - 360) if longitude > 180 else longitude
# example useage:
longitudes_0_360 = [0, 90, 180, 270, 360]
longitudes_neg180_180 = [convert_longitude(lon) for lon in longitude]

print(longitude_neg180_180)